# Quantum Topological Data Analysis on 5 Qubits of IBM's Backend

Since the discovery of Shor’s factoring quantum algorithm in 1994, Theoretical Computer Scientists have been in a race to develop more quantum algorithms for potential future quantum computers (A Brief History of Quantum Computing, S. Bone & M. Castro). Today, “Quantum Algorithm Zoo” cites over 270 quantum algorithm papers (Quantum Algorithm Zoo). 

In 2009, the discovery of the HHL quantum algorithm extended the potential applicability of future quantum computers beyond just accelerated simulation of quantum physics and chemistry to accelerated computation in machine learning (Harrow, Hassidam, Lloyd, 2009). Classically, solving a linear system, Ax ⃗= b ⃗, where A is a N x N sparse matrix with a condition number κ, solves for x ⃗ in O(N√κ) using the best known classical algorithm, whereas HHL quantum algorithm solves for x ⃗ in poly(log N, κ) time (Harrow, Hassidam, Lloyd, 2009). That claim is yet defeated when accounting for the expensive load required for preparing the vector state b ⃗ of the linear system from a quantum RAM (S. Aaronson).

Provided here is a potential analytic workaround, to the quantum RAM expensive load problem; potentially, memory requirement of the quantum RAM may be exponentially suppressed using a quantum algorithm applicable to a different data analytic tool. This alternative is the quantum algorithm for topological data analysis (QTDA), founded by Seth Lloyd, Silvano Garnerone, and Paolo Zanardi (2015), which, with techniques of topology, offers a way to extract information from noisy, unstructured data. The quantum algorithm applied to TDA achieves an exponential suppression of the memory requirement for the quantum RAM, requiring just O(n^2) bits, as opposed to O(2^n) bits as in HHL (S. Aaronson). 

Lloyd, Garnerone, and Zanardo discovery of QTDA are algebraic discoveries. Such algorithm needs to be examined numerically with software and simulators. Here, development is done using ProjectQ, an open-source software framework for quantum computing. This framework utilizes IBM’s quantum processing backend for processing. Following a recent experiment (Huang et al 2018), we use ProjectQ to replicate the five-qubits circuit experiment that analyzes the topological features of Betti numbers for a network of three data points.

In [ ]:
from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.ops import H, Measure, CNOT, X, Z, Rz
from projectq.meta import Dagger, Control
import projectq.setups.ibm
from projectq.backends import IBMBackend

First, need to define a function for the simplicial complex construction of the three data points. a, b, and c are qubits for the three data points, d is an ancilla qubit, and e is the register qubit.

In [ ]:
def simplicial_complex_setup(eng):
    a = eng.allocate_qubit()
    b = eng.allocate_qubit()
    c = eng.allocate_qubit()
    d = eng.allocate_qubit()
    e = eng.allocate_qubit()

    H | a
    X | c
    CNOT | (a, b)
    X | b

    return a, b, c, d, e

Need a function that calls simplicial_complex_setup to provide initial superposition pure state construction of the simplicial complex for the 3 data points. Then, a mixture state construction must be performed. Last, phase estimation is performed.

In [ ]:
def qtda(eng):
    #simplicial complex construction
    a, b, c, d, e = simplicial_complex_setup(eng)
    
    #mixture state construction
    CNOT | (b, d)
    
    #phase estimation
    CNOT | (c, e)
    
    return a, b, c, d, e

Point of entry. First, initialize MainEngine to run on IBM backend, then call qtda to run the QTDA algorithm, and last, run measurement on register qubit. 

In [ ]:
if __name__ == "__main__":
    # create a main compiler engine with IBM's backend:
    eng = MainEngine(IBMBackend())

    # run the qtda algorithm:
    a, b, c, d, e = qtda(eng)
    
    # measure
    Measure | e

    eng.flush()
    # print the result:
    print("Measured e: {}".format(int(e)))